In [ ]:
import yaml
import os
import xarray as xr
import fsspec
import pandas as pd

def convert_time_index_to_datetime(ds, dim):
    return ds.assign_coords({dim: ds.indexes[dim].to_datetimeindex()})

In [ ]:
with open("rundirs.yml") as f:
    rundirs = yaml.safe_load(f)

BUCKET = 'gs://vcm-ml-data/testing-2020-02/' 
metrics = {}

for rundir in rundirs:
    path = os.path.join(BUCKET, f"{rundir['name']}.nc")
    with fsspec.open(path, "rb") as f:
        metrics[rundir['name']] = xr.open_dataset(f, engine='h5netcdf').load()

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
def get_ts(ds):
    return ds.drop([key for key in ds if set(ds[key].dims) != {'time'}])

In [ ]:
time_series = {key: convert_time_index_to_datetime(get_ts(ds), 'time') for key, ds in metrics.items()}

In [ ]:
def holomap_filter(time_series, varfilter):
    p = hv.Cycle('Colorblind')
    hmap = hv.HoloMap(kdims=['variable', 'run'])
    for run, ds in time_series.items():
        for varname in ds:
            if varfilter in varname:
                try:
                    v = ds[varname].load()
                except:
                    pass
                else:
                    if run.endswith('baseline'):
                        style= 'dashed'
                    else:
                        style = 'solid'
                    long_name = ds[varname].long_name
                    hmap[(long_name, run)] = hv.Curve(v, label=varfilter).options(line_dash=style, color=p)
    return hmap

# RMS Errors

In [ ]:
%%opts Curve [width=800, height=600] {+framewise}
hmap = holomap_filter(time_series, 'rms')
hmap.overlay('run')

# Global Averages

In [ ]:
%%opts Curve [width=800, height=600] {+framewise}
hmap = holomap_filter(time_series, 'global_avg')
hmap.overlay('run')